In [1]:
import pynq
import numpy as np
from time import sleep
u2 = pynq.Device.devices[1]
ol = pynq.Overlay("krnl_dram_rw.xclbin", device=u2)
krnl = ol.krnl_dram_rw_1
krnl_map = krnl.register_map

In [2]:
ol?

In [3]:
print(krnl_map)

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, AUTO_RESTART=0, AP_CONTINUE=0),
  wr_en = Register(value=0),
  data_i = Register(value=0),
  data_o = Register(value=0),
  dummy = Register(value=0)
}


In [4]:
krnl.signature

<Signature (wr_en: 'unsigned int', data_i: 'void*', data_o: 'void*', dummy: 'void*')>

In [5]:
def loop_till_fail():
    count = 0
    time = 0.01
    flag = True
    while flag:
        out = pynq.allocate(shape=(2048,2048), dtype=np.uint32, target=ol.bank0)
        gold = np.random.randint(low=0, high=100, size=(2048,2048), dtype=np.uint32)
        out[:] = gold
        out.sync_to_device()
        sleep(time)
        out.sync_from_device()

        if (np.array_equal(out, gold)):
            del(out)
            del(gold)
            time += 0.0001
            count += 1
        else:
            del(out)
            del(gold)
            flag = False
            #print(f"Corrupted on loop {count} in {time} seconds.")
    return time, count

In [6]:
results = []
loops = []
for _ in range(20):
    time, count = loop_till_fail()
    results.append(time)
    loops.append(count)

avg_time = sum(results) / len(results)
avg_loops = sum(loops) / len(loops)
print(f"Avg of {count} loops in {avg_time}")

Avg of 108 loops in 0.022649999999999927


In [7]:
data_o = pynq.allocate(shape=(1,), dtype=np.uint32, target=ol.bank0)
data_i = pynq.allocate(shape=(1,), dtype=np.uint32, target=ol.plram)
dummy = pynq.allocate(shape=(1,), dtype=np.uint32, target=ol.plram)

data_i[0] = 10

data_o.sync_to_device()
#data_i.sync_to_device()

krnl.call(1, data_i, data_o, dummy)

#data_i.sync_from_device()
data_o.sync_from_device()

print(f"{data_i}: data_i")
print(f"{data_o}: data_o")

del(data_o)
del(data_i)
del(dummy)

[10]: data_i
[0]: data_o


In [10]:
def print_ip_dict(d, indent=0):
    for key, value in d.items():
        if isinstance(value, dict):
            print('  ' * indent + str(key))
            print_ip_dict(value, indent+1)
        else:
            print('  ' * (indent+1) + f"{key}: {value}")

In [11]:
print_ip_dict(ol.ip_dict["krnl_dram_rw_1"])

  phys_addr: 25231360
  addr_range: 4168
  type: xilinx.com:hls:krnl_dram_rw:1.0
  hw_control_protocol: ap_ctrl_chain
  fullpath: krnl_dram_rw_1
registers
  CTRL
      address_offset: 0
      access: read-write
      size: 4
      description: OpenCL Control Register
      type: unsigned int
      id: None
    fields
      AP_START
          access: read-write
          bit_offset: 0
          bit_width: 1
          description: Start the accelerator
      AP_DONE
          access: read-only
          bit_offset: 1
          bit_width: 1
          description: Accelerator has finished - cleared on read
      AP_IDLE
          access: read-only
          bit_offset: 2
          bit_width: 1
          description: Accelerator is idle
      AP_READY
          access: read-only
          bit_offset: 3
          bit_width: 1
          description: Accelerator is ready to start next computation
      AUTO_RESTART
          access: read-write
          bit_offset: 7
          bit_width: 1
   